# 4.7 Deriving New Variables

### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# Setting path to Achievement 4 master folder
path = r'/Users/jeremyobach/Documents/Data Analytics/CareerFoundry/Achievement 4- Python for DA/DEC23 Instacart Basket Analysis - MASTER FOLDER'
#checking path set correctly
path

'/Users/jeremyobach/Documents/Data Analytics/CareerFoundry/Achievement 4- Python for DA/DEC23 Instacart Basket Analysis - MASTER FOLDER'

In [3]:
#importing ords_prods_merge, combined df from 4.6, as ords_prods_merge

ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data','ords_prods_merge.pkl'))

### Creating subset of ords_prods_merge with "only" first million rows as df_short


In [4]:
df_short = ords_prods_merge[:1000000]

###  Creating a new variable to designate low, mid, high range product based on price.


In [5]:
def price_label(row):
    
    if row['prices'] <= 5:
            return 'Low-range product'
    elif (row['prices'] >5) and (row['prices']<=15):
            return 'Mid-range product'
    elif row['prices'] > 15:
            return 'High range'
    else: return 'Not enough data'

In [6]:
df_short['price_range'] = df_short.apply(price_label, axis=1)

# Recieved caveat warning message that loc() method could be used instead

/var/folders/12/fz0w81fd1mb1hklwgzyg3bx00000gn/T/ipykernel_5851/3886719338.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_short['price_range'] = df_short.apply(price_label, axis=1)


In [7]:
df_short['price_range'].value_counts(dropna = False)

# Unclear why the lesson said I would get no High range products...

price_range
Mid-range product    756450
Low-range product    243550
Name: count, dtype: int64

In [8]:
df_short['prices'].max()

14.8

### Uh oh. According to the lesson, this indicates some miscalculation on a previous step, maybe in the creation of the ords_prods_merge dataframe. 

#### There should be zero high range products, with a max price of 14.80

## Update: now getting results in line with the examples, having changed merge type in 4.6 from 'left' to 'inner'

### Using loc() method on the unabridged version of the ords_prod_merge df

In [9]:
ords_prods_merge.loc[ords_prods_merge['prices'] > 15, 'price_range_loc'] = 'High range product'

/var/folders/12/fz0w81fd1mb1hklwgzyg3bx00000gn/T/ipykernel_5851/1810973517.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'High range product' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ords_prods_merge.loc[ords_prods_merge['prices'] > 15, 'price_range_loc'] = 'High range product'


In [10]:
ords_prods_merge.loc[(ords_prods_merge['prices'] <= 15) & (ords_prods_merge['prices'] > 5), 'price_range_loc'] = 'Mid range product' 

In [11]:
ords_prods_merge.loc[ords_prods_merge['prices'] <= 5, 'price_range_loc'] = 'Low range product'

In [12]:
ords_prods_merge['price_range_loc'].value_counts(dropna = False)

price_range_loc
Mid range product     21860860
Low range product     10126321
High range product      417678
Name: count, dtype: int64

### We can see in the full dataset, as opposed to df_short with only the first million entries, that we have 417,678 high range products. 

In [13]:
ords_prods_merge['orders_day_of_week'].value_counts(dropna = False)

orders_day_of_week
0    6204182
1    5660230
6    4496490
2    4213830
5    4205791
3    3840534
4    3783802
Name: count, dtype: int64

### Per project brief, 0 = Saturday, 1 Sunday and so on...
#### Saturday the busiest day, 4=Wednesday for the least busy day
### Creating "Busiest Day" list, which we will map to a column

In [14]:
result = []

for value in ords_prods_merge["orders_day_of_week"]:
  if value == 0:
    result.append("Busiest day")
  elif value == 4:
    result.append("Least busy")
  else:
    result.append("Regularly busy")

In [15]:
result

['Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Least busy',
 'Least busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Least busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Least busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Least busy',
 'Least busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Least busy',
 'Regularly busy',
 'Busiest day',
 'Regularly busy',
 'Reg

In [16]:
# Creating a new column within ords_prods_merge, setting equal to result

ords_prods_merge['busiest_day'] = result

### Printed frequency of new column, matches the counts from orders_day_of_week printed before


In [31]:
ords_prods_merge['busiest_day'].value_counts(dropna = False)

busiest_day
Regularly busy    22416875
Busiest day        6204182
Least busy         3783802
Name: count, dtype: int64

###  UPDATE: REMOVING  'BUSIEST DAY'

In [37]:
ords_prods_merge.columns

Index(['order_id', 'user_id', 'order_number', 'orders_day_of_week',
       'orders_hour_of_day', 'days_since_prior_order', 'first_order',
       'product_id', 'add_to_cart_order', 'reordered', '_merge',
       'product_name', 'aisle_id', 'department_id', 'prices',
       'price_range_loc', 'busiest_day', 'busiest_days',
       'busiest_period_of_day'],
      dtype='object')

In [41]:
ords_prods_merge = ords_prods_merge.drop(columns =['busiest_day'])

In [43]:
ords_prods_merge.columns

Index(['order_id', 'user_id', 'order_number', 'orders_day_of_week',
       'orders_hour_of_day', 'days_since_prior_order', 'first_order',
       'product_id', 'add_to_cart_order', 'reordered', '_merge',
       'product_name', 'aisle_id', 'department_id', 'prices',
       'price_range_loc', 'busiest_days', 'busiest_period_of_day'],
      dtype='object')

# Task:

## 1. "price_label" and "busiest_day" columns created

## 2. Per client's request, creating "Busiest Days" column, to designate two busiest and two slowest days

### Per value counts from before, second busiest day is 1=Sunday, and second slowest day is 3=Tuesday

In [44]:
result = []

for value in ords_prods_merge["orders_day_of_week"]:
  if value in [0, 1]:
    result.append("Busiest days")
  elif value in [4, 3]:
    result.append("Slowest days")
  else:
    result.append("Regularly busy")

In [45]:
result

['Regularly busy',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Regularly busy',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Slowest days',
 'Busiest days',
 'Regularly busy',
 'Regularly busy',
 'Busiest days',
 'Busiest days',
 'Regularly busy',
 'Regularly busy',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Regularly busy',
 'Regularly busy',
 'Busiest days',
 'Regularly busy',
 'Regularly busy',
 'Busiest days',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Slowest days',
 'Regularly busy',
 'Slowest days',
 'Busiest days',
 'Busiest days',
 'Regularly busy',
 'Slowest days',
 'Slowest days',
 'Regularly busy',
 'Regularly busy',
 'Busiest days',
 'Busiest days',
 'Regularly busy',
 'Busiest days',
 'Busiest days',
 'Slowest days',
 'Regularly busy',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest 

In [46]:
ords_prods_merge['busiest_days'] = result

In [47]:
ords_prods_merge['busiest_days'].value_counts(dropna = False)

busiest_days
Regularly busy    12916111
Busiest days      11864412
Slowest days       7624336
Name: count, dtype: int64

## 3.  Sums for two busiest, two slowest days check out as expected.

## 4.  Identify busiest hours of the day, group into periods of time lavbeled "Most orders", "Average orders", and "Fewest orders".

### Will have to make judgement call based on the data returned, decide how many hours ought to be grouped together.

In [48]:
ords_prods_merge['orders_hour_of_day'].value_counts(dropna =False)

orders_hour_of_day
10    2761760
11    2736140
14    2689136
15    2662144
13    2660954
12    2618532
16    2535202
9     2454203
17    2087654
8     1718118
18    1636502
19    1258305
20     976156
7      891054
21     795637
22     634225
23     402316
6      290493
0      218769
1      115700
5       87961
2       69375
4       53242
3       51281
Name: count, dtype: int64

### Based on this distribution, I'm going to group busiest as the 6 busiest hours, from 1000-1500, and the fewest as the 6 slowest hours, 0000-0500.

In [49]:
result = []

for value in ords_prods_merge["orders_hour_of_day"]:
  if value in [10,11,12,13,14,15]:
    result.append("Most orders")
  elif value in [0,1,2,3,4,5]:
    result.append("Fewest orders")
  else:
    result.append("Average orders")

In [50]:
result

['Average orders',
 'Average orders',
 'Most orders',
 'Average orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Most orders',
 'Average orders',
 'Most orders',
 'Average orders',
 'Most orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Most orders',
 'Average orders',
 'Most orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Average orders',


In [51]:
ords_prods_merge["busiest_period_of_day"] = result

## 5. Frequency of Busiest_period_of_day column:

In [52]:
ords_prods_merge["busiest_period_of_day"].value_counts(dropna = False)

busiest_period_of_day
Most orders       16128666
Average orders    15679865
Fewest orders       596328
Name: count, dtype: int64

In [53]:
ords_prods_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge_derived.pkl'))

## 6. Check for Notebook structure, cleanliness

### New version of ords_prods_merge with new busiest period, busiest time columns. Calling it ords_prods_merge_derived.pkl

In [54]:
ords_prods_merge.shape

(32404859, 18)

In [55]:
ords_prods_merge.head(25)

order_id  user_id  order_number  orders_day_of_week  orders_hour_of_day  \
0    2539329        1             1                   2                   8   
1    2398795        1             2                   3                   7   
2     473747        1             3                   3                  12   
3    2254736        1             4                   4                   7   
4     431534        1             5                   4                  15   
5    3367565        1             6                   2                   7   
6     550135        1             7                   1                   9   
7    3108588        1             8                   1                  14   
8    2295261        1             9                   1                  16   
9    2550362        1            10                   4                   8   
10   2968173       15            15                   1                   9   
11   1870022       15            17                   2                  16   
12   1911383       15            18                   2                  11   
13   2715276       15            21                   1                   9   
14    487368       15            22                   1                  10   
15   2293453       19             2                   5                  14   
16   1973799       19             5                   6                  12   
17    532817       19             7                   4                  17   
18   1573906       21            10                   3                  10   
19   1593000       31            10                   3                   8   
20   2231262       31            17                   3                  11   
21   2580647       43             6                   4                  16   
22   2187180       43             9                   4                  12   
23   2497897       52             1                   1                   9   
24   1318871       52             2                   1                  10   

    days_since_prior_order  first_order  product_id  add_to_cart_order  \
0                      NaN         True         196                  1   
1                     15.0        False         196                  1   
2                     21.0        False         196                  1   
3                     29.0        False         196                  1   
4                     28.0        False         196                  1   
5                     19.0        False         196                  1   
6                     20.0        False         196                  1   
7                     14.0        False         196                  2   
8                      0.0        False         196                  4   
9                     30.0        False         196                  1   
10                     7.0        False         196                  2   
11                     8.0        False         196                  6   
12                     7.0        False         196                  1   
13                     7.0        False         196                  1   
14                    14.0        False         196                  1   
15                     6.0        False         196                  3   
16                     8.0        False         196                 15   
17                     6.0        False         196                  1   
18                     6.0        False         196                  2   
19                     7.0        False         196                 17   
20                     8.0        False         196                 14   
21                     4.0        False         196                  9   
22                     3.0        False         196                  1   
23                     NaN         True         196                  4   
24                     7.0        False         196                  5   

    reordered _merge pro